# Loading and Using Finetuned Models

This notebook contains simplified code to load a fine-tuned classification model, which in this case is based on GPT-2 Medium (355M). The detailed code can be found in [Notebook 05-finetuning-text-classification.ipynb](05-finetuning-text-classification.ipynb)

In [1]:
from importlib.metadata import version

pkgs = [
    "tiktoken",
    "torch"
]

for p in pkgs:
    print(f"{p} version: {version(p)}")

tiktoken version: 0.9.0
torch version: 2.7.0


In [2]:
from pathlib import Path

ft_model_path = Path("models/review_classifier_gpt2_medium.pth")

if not ft_model_path.exists():
    print(f"Couldn't find {ft_model_path}.\n" 
           "Re-run NB 05-finetuning-text-classification to fine tune and save a model.")

In [ ]:
from utils.components import GPTModel

BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

PosixPath('models/review_classifier_gpt2_medium.pth')